# Advent of Code 2019
## Day 11: Space Police
(https://adventofcode.com/2019/day/11)

----
## Part 1
----

In [1]:
import numpy as np
import copy

## Read in data

In [33]:
with open("inputs\\painting_program.txt") as _file:
    for line in _file:
        program = [int(x) for x in line.split(',')] 
        
program = copy.deepcopy(program) + 1000 * [0]

## Intcode computer

In [35]:
def run_intcode_computer(program, int_input):

    position = 0
    output = []
    opcode = ''
#    program = copy.deepcopy(program) + 1000 * [0]
    
    num_params = [0, 3, 3, 1, 1, 2, 2, 3, 3, 1] # each element maps to an opcode

    relative_base = 0
    
    while True:

        opcode_str = (5 - len(str(program[position]))) * '0' + str(program[position])
        opcode = int(opcode_str[-2:])

        # opcode 99 is terminate, takes 0 parameters    
        if opcode == 99:
            break
        
        modes = [0, int(opcode_str[-3]), int(opcode_str[-4]), int(opcode_str[-5])]
        
        arg_pos = [0] # Argument position, start list with a 0 pad
        for i in range(1, num_params[opcode] + 1):
            if i <= num_params[opcode]:
                # position mode
                if modes[i] == 0:  
                    arg_pos.append(program[position + i])
                # immediate mode
                elif modes[i] == 1: 
                    arg_pos.append(position + i)
                # relative mode
                elif modes[i] == 2:
                    arg_pos.append(relative_base + program[position + i])

        #print(opcode_str, position)
                    
        # Set next opcode position
        position += num_params[opcode] + 1
        
        # opcode 1 is addition, takes 3 parameters
        if opcode == 1:
            program[arg_pos[3]] = program[arg_pos[1]] + program[arg_pos[2]]

        # opcode 2 is multiplication, takes 3 parameters
        elif opcode == 2:
            program[arg_pos[3]] = program[arg_pos[1]] * program[arg_pos[2]]

        # opcode 3 is input, takes 1 parameter
        elif opcode == 3:  
            program[arg_pos[1]] = int_input.pop(0)
            
        # opcode 4 is output, takes 1 parameter
        elif opcode == 4:
            yield program[arg_pos[1]]
            #output.append(program[arg_pos[1]])
            
        # opcode 5, jump if true, takes 2 parameters
        elif opcode == 5:
            if program[arg_pos[1]] == True:
                position = program[arg_pos[2]]
        
        # opcode 6, jump if false, takes 2 parameters
        elif opcode == 6:
            if program[arg_pos[1]] == False:
                position = program[arg_pos[2]]
        
        # opcode 7, less than, takes 3 parameters
        elif opcode == 7:
            if program[arg_pos[1]] < program[arg_pos[2]]:
                program[arg_pos[3]] = 1
            else:
                program[arg_pos[3]] = 0

        # opcode 8, equals, takes 3 parameters
        elif opcode == 8:
            if program[arg_pos[1]] == program[arg_pos[2]]:
                program[arg_pos[3]] = 1
            else:
                program[arg_pos[3]] = 0
                
        # opcode 9, adjust relative base, takes 1 parameter
        elif opcode == 9:
            relative_base += program[arg_pos[1]]     
    
    return output

## Painting Robot

In [36]:
def change_pos(dir, x, y):
    """
    Change position (always incrementing by 1), given current position and direction
    robot is facing.
    """
    if dir == 0: 
        y += 1 # move up
    elif dir == 1:
        x += 1 # move right
    elif dir == 2:
        y -= 1 # move down
    elif dir == 3:
        x -= 1 # move left

    return x, y

def change_dir(current_dir, instruct):
    """
    Change direction of painting robot based on instruction returned by the Intcode
    computer.
    """
    if instruct == 0: # turn left
        return (current_dir - 1) % 4            
    elif instruct == 1: #turn right
        return (current_dir + 1) % 4


def run_painting_robot(program, initial_color):

    
    x, y = 0, 0 # start off at origin
    panels = {(x, y) : initial_color} 
    dir = 0 # start off facing up
    dir_dict = {0 : 'up', 1 : 'right', 2 : 'down', 3 : 'left'}

    int_input = []
    computer = run_intcode_computer(program, int_input)    
    
    while True:
#         print('top')
#         print('initial direction: ', dir_dict[dir])
#         print('initial position: ', x, y)
        
        
        # if panel (x, y) hasn't been visited yet, set default color to black (0)
        int_input.append(panels.get((x, y), 0)) 
#         print('initial color: ', int_input)
        
        try:
            
            # Paint current position
            panels[(x, y)] = next(computer)
#             print('new color: ', int(panels[(x, y)]))
            
            # Change direction
            dir = change_dir(dir, next(computer))
#             print('new direction: ', dir_dict[dir])
            
            # Change position
            x, y = change_pos(dir, x, y)
#             print('new position: ', x, y)

#             print('bottom \n')
        
        except StopIteration:
            return panels


In [37]:
panels = run_painting_robot(program, 0)

In [38]:
len(panels)

1894

## Part 2

In [48]:
with open("inputs\\painting_program.txt") as _file:
    for line in _file:
        program = [int(x) for x in line.split(',')] 
        
program = copy.deepcopy(program) + 1000 * [0]

In [49]:
panels = run_painting_robot(program, 1)

In [50]:
# Source: https://github.com/aspittel/advent-of-code/blob/master/2019/dec-11/script.py

data = [[" " for _ in range(50)] for _ in range(6)]
for x, y in panels.keys():
    color = panels[(x, y)]
    # KP: I don't understand why x and y are reversed and why need to use abs(y)
    data[abs(y)][x] = " " if color == 0 else "*"
for row in data:
    print(''.join(row))

   ** *  * **** *    ****   ** ***  *  *          
    * * *     * *       *    * *  * *  *          
    * **     *  *      *     * ***  ****          
    * * *   *   *     *      * *  * *  *          
 *  * * *  *    *    *    *  * *  * *  *          
  **  *  * **** **** ****  **  ***  *  *          


## Test Generators

In [12]:
def gen(foo, n):
    while True: # Start an infinite loop
        print(foo)
        yield n.pop() + 1

In [13]:
n = []

In [14]:
g = gen(2, n) 
# Once you link a list input to a generator function, you can pop from the list inside the
# generator function and updating the list with append statements.

In [15]:
for i in range(10):
    n.append(i)
    print(i, n)
    bar = next(g)
    print(bar)

0 [0]
2
1
1 [1]
2
2
2 [2]
2
3
3 [3]
2
4
4 [4]
2
5
5 [5]
2
6
6 [6]
2
7
7 [7]
2
8
8 [8]
2
9
9 [9]
2
10


In [16]:
next(g)

2


IndexError: pop from empty list

In [17]:
data = [[" " for _ in range(50)] for _ in range(6)]

In [18]:
data

[[' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '],
 [' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' '],
 [' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' ',
  ' 

In [19]:
for row in data:
            print(''.join(row))